In [6]:
input_dir = "/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/ASE_readCount/input_count/"
ip_dir = "/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/ASE_readCount/ip_count/"
###
result_dir = "/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/ASE_readCount/read_matrix/"
ip_list = glob.glob("%s/*.readcounts.txt" % ip_dir)

In [2]:
map_file = "/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/ASE_readCount/total_Input_vs_IP.txt"
map_dict = {}
df_map = pd.read_table(map_file)
map_dict = dict(zip(df_map['IP'], df_map['Input']))

In [3]:
# filter based on read counts > 5;
# file_type == "ip" or "input"
def preprocess_count(in_file, file_type): 
    df = pd.read_table(in_file)
    df["ID"] = df["contig"] + ":" + df["position"].astype(str)
    df_sub = df[["ID", "refCount", "altCount"]]
    df_sub = df_sub[(df_sub['refCount']>5) & (df_sub['altCount']>5)]
    df_sub.columns = ["ID", "refCount_%s" % file_type, "altCount_%s" % file_type]
    return df_sub

In [5]:
def generate_totalNum_dict():
    df_map = pd.read_table("/home/galaxy/project/alleleSpecific_analysis/data/bam_wasp/total_Input_vs_IP.txt")
    map_dict = dict(zip(df_map['IP'], df_map['Input']))
    #
    summary_file = "/home/galaxy/project/alleleSpecific_analysis/data/bam_wasp/stat/AllSamples.bam_stat.txt"
    df_stat = pd.read_table(summary_file, header=None)
    number_dict = dict(zip(df_stat.iloc[:,0].tolist(), df_stat.iloc[:,1].tolist()))
    #
    totalNum_dict = {}
    for sample in number_dict.keys():
        if sample in map_dict:
            # ip_sample: [ip_num, input_sample, input_num]
            totalNum_dict[sample] = [number_dict[sample], map_dict[sample], number_dict[map_dict[sample]]]
    return totalNum_dict
totalNum_dict = generate_totalNum_dict()
print(totalNum_dict["CRR042312"]) # [a, "CRR042313", b]

[109697553, 'CRR042313', 59713070]


In [7]:
for ip_count in ip_list:
    ip_prefix = os.path.basename(ip_count).split(".")[0]
    result_file = os.path.join(result_dir, "%s.matrix" % ip_prefix)
    input_prefix = map_dict[ip_prefix]
    ###
    input_count = os.path.join(input_dir, input_prefix + ".readcounts.txt")
    df_input, df_ip = preprocess_count(input_count, "input"), preprocess_count(ip_count, "ip")
    df = df_input.merge(df_ip, on="ID")
    ip_totalNum, input_totalNum = int(totalNum_dict[ip_prefix][0]), int(totalNum_dict[ip_prefix][2])
    # normalized by total number
    df["refCount_input_norm"] = df["refCount_input"] / input_totalNum
    df["altCount_input_norm"] = df["altCount_input"] / input_totalNum
    df["refCount_ip_norm"] = df["refCount_ip"] / ip_totalNum
    df["altCount_ip_norm"] = df["altCount_ip"] / ip_totalNum
    # calculate m6A abundance
    df["refCount_abundance"] = df["refCount_ip_norm"] / df["refCount_input_norm"]
    df["altCount_abundance"] = df["altCount_ip_norm"] / df["altCount_input_norm"]
#     print(df.head())
    df.to_csv(result_file, sep="\t", index=False)